In [1]:
# importando librerias
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import collections
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost
import math as mt
import warnings

# Ploting styles
# styles: 'fivethirtyeight', 'classic', 'ggplot', 'seaborn-notebook'
# styles: 'seaborn-poster', 'bmh', 'grayscale', 'seaborn-whitegrid'
matplotlib.style.use('bmh')
%matplotlib inline
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#print(plt.style.available)
warnings.filterwarnings("ignore")

In [2]:
#propios utilitarios
from utils import *
data_train = pd.read_csv('data/01dataBaseTrainTrxRec.csv', **set_parameter_csv)
data_train.head()

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto
0,2017-01-29 00:00:00,7649,138.0,43629,1,176.0,1,0.014072
1,2016-12-01 00:00:00,24604,75.0,4326,0,81.0,1,0.001667
2,2017-06-01 00:00:00,15289,75.0,4326,0,81.0,1,0.000127
3,2017-09-22 00:00:00,5190,110.0,59776,1,156.0,1,0.001167
4,2017-05-05 00:00:00,16635,75.0,31043,0,81.0,1,0.047386


In [3]:
data_train.sort_values(by=['codCliente','codEstab']).head()

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto
25041,2017-05-24 00:00:00,1,105.0,39,1,147.0,1,0.084590
522851,2017-03-07 00:00:00,1,105.0,39,1,147.0,1,0.098481
743922,2017-08-20 00:00:00,1,105.0,39,1,147.0,1,0.018126
1272804,2017-05-13 00:00:00,1,105.0,39,1,147.0,1,0.018183
816568,2016-12-22 00:00:00,1,72.0,1194,1,147.0,1,0.031735


In [4]:
pd.concat([
    pd.DataFrame(data_train.groupby(['codCliente'])['ratingMonto'].sum()),
    pd.DataFrame(data_train.groupby(['codCliente'])['ratingMonto'].mean()),
    pd.DataFrame(data_train.groupby(['codCliente'])['ratingMonto'].count())],
    axis=1
)[:5]

,ratingMonto,ratingMonto,ratingMonto
codCliente,,,
1,0.863403,0.023983,36
2,0.540837,0.004545,119
3,0.582373,0.044798,13
4,0.774747,0.004280,181
5,0.621249,0.015152,41


In [5]:
pd.concat([
    pd.DataFrame(data_train.groupby(['codCliente', 'codEstab'])['ratingMonto'].sum()),
    pd.DataFrame(data_train.groupby(['codCliente', 'codEstab'])['ratingMonto'].mean()),
    pd.DataFrame(data_train.groupby(['codCliente', 'codEstab'])['ratingMonto'].count())],
    axis=1
)[:5]

ratingMonto  ratingMonto  ratingMonto
codCliente codEstab                                       
1          39           0.219380     0.054845            4
           1194         0.031735     0.031735            1
           1911         0.119995     0.119995            1
           2399         0.009543     0.009543            1
           2425         0.024790     0.012395            2

In [6]:
data_perfil = pd.read_csv('data/02dataBasePerfilRec.csv', **set_parameter_csv)
data_perfil.head()

,codCliente,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4
0,24041,Rango2,Rango2,1,0,360.0,Rango1,Rango1,Rango1,1,NaN,NaN,NaN,NaN
1,9040,Rango4,Rango6,1,1,260.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1
2,18233,Rango2,Rango1,1,1,273.0,Rango1,Rango2,Rango1,1,Rango2,Rango1,Rango1,Rango1
3,7226,Rango5,Rango3,0,1,241.0,Rango1,Rango2,Rango1,1,Rango1,Rango2,Rango1,Rango2
4,21617,Rango3,Rango2,1,1,268.0,Rango1,Rango1,Rango1,0,NaN,NaN,NaN,NaN


In [7]:
data_perfil.sort_values(by=['codCliente']).head()

,codCliente,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4
1599,1,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
25805,2,Rango6,Rango2,1,1,241.0,Rango1,Rango3,Rango1,0,Rango6,Rango1,Rango1,Rango1
700,3,Rango6,Rango5,0,1,253.0,Rango1,Rango2,Rango2,0,Rango6,Rango2,Rango1,Rango2
19987,4,Rango6,Rango1,1,0,212.0,Rango1,Rango1,Rango2,0,Rango1,Rango1,Rango1,Rango1
29861,5,Rango6,Rango1,1,1,271.0,Rango1,Rango1,Rango3,1,Rango2,Rango1,Rango5,Rango1


In [8]:
data_train.shape, data_perfil.shape, data_train['codCliente'].unique().shape

((1591617, 8), (30000, 14), (30000,))

In [9]:
TrainTot = pd.merge(data_train, data_perfil, on='codCliente')
TrainTot.head()

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4
0,2017-01-29 00:00:00,7649,138.0,43629,1,176.0,1,0.014072,Rango4,Rango2,1,1,281.0,Rango1,Rango2,Rango1,1,Rango3,Rango1,Rango1,Rango1
1,2017-05-07 00:00:00,7649,75.0,830,1,157.0,1,0.000821,Rango4,Rango2,1,1,281.0,Rango1,Rango2,Rango1,1,Rango3,Rango1,Rango1,Rango1
2,2016-12-20 00:00:00,7649,75.0,829,1,175.0,1,0.001220,Rango4,Rango2,1,1,281.0,Rango1,Rango2,Rango1,1,Rango3,Rango1,Rango1,Rango1
3,2017-02-16 00:00:00,7649,114.0,21603,1,175.0,1,0.000985,Rango4,Rango2,1,1,281.0,Rango1,Rango2,Rango1,1,Rango3,Rango1,Rango1,Rango1
4,2017-08-30 00:00:00,7649,110.0,46216,1,203.0,1,0.003635,Rango4,Rango2,1,1,281.0,Rango1,Rango2,Rango1,1,Rango3,Rango1,Rango1,Rango1


In [10]:
# Eliminacion de columnas fecha y Ctdrx
TrainTot.drop(['fechaOper', 'ctdTrx'], axis=1, inplace=True)
TrainTot.shape, TrainTot['codCliente'].unique().shape, TrainTot.columns

((1591617, 19),
 (30000,),
 Index(['codCliente', 'codGiro', 'codEstab', 'flagLimaProvEstab', 'ubigeoEstab',
        'ratingMonto', 'rangoEdad', 'rangoIngreso', 'flagGenero',
        'flagLimaProvCliente', 'ubigeoCliente', 'rangoCtdProdAct',
        'rangoCtdProdPas', 'rangoCtdProdSeg', 'flagBxi', 'saldoTcEntidad1',
        'saldoTcEntidad2', 'saldoTcEntidad3', 'saldoTcEntidad4'],
       dtype='object'))

In [11]:
TrainTot['ratingMonto'].describe()

count    1.591617e+06
mean     1.318383e-02
std      3.236438e-02
min      2.984905e-06
25%      1.598366e-03
50%      4.199563e-03
75%      1.152572e-02
max      1.000000e+00
Name: ratingMonto, dtype: float64

In [12]:
#Carga de CSV a entregar soluciones
BaseSub = pd.read_csv('data/03dataBaseTestRec.csv')
BaseSub.head()

,codClienteCodEstab
0,100001675
1,1000217838
2,1000219843
3,1000231391
4,100032061


In [13]:
BasePred = pd.read_csv('data/05dataBaseTestKeyRec.csv')
BasePred.sort_values(by=['codCliente', 'codEstab']).head()

,codCliente,codEstab
28161,1,39
244120,1,3843
40138,1,6314
407005,1,12767
378968,1,26260


In [31]:
BaseSub.shape, BasePred.shape, BaseSub['codClienteCodEstab'].unique().shape

((467203, 1), (467203, 2), (467203,))

In [29]:
print(data_train.shape)
combine_client_estab = list((data_train["codCliente"].map(str) + data_train["codEstab"].map(str)).unique())
print(len(combine_client_estab))
set_client_estab = set(combine_client_estab)
print(len(set_client_estab))

(1591617, 8)
901865
901865


In [40]:
set_est_client_pred = set(list(BaseSub['codClienteCodEstab'].map(str).unique()))
len(set_est_client_pred)

467203

In [45]:
len(set_client_estab & set_est_client_pred), len(set_client_estab | set_est_client_pred)

(194237, 1174831)

In [48]:
len(set_est_client_pred - set_client_estab)

272966

## Analisis de su eliminacion de duplicados

In [20]:
FeatEstab = data_train.drop(['codCliente','ratingMonto', 'fechaOper', 'ctdTrx'], axis=1)
print(FeatEstab.shape, FeatEstab.columns)
FeatEstab.sort_values(by=['codEstab']).head()

(1591617, 4) Index(['codGiro', 'codEstab', 'flagLimaProvEstab', 'ubigeoEstab'], dtype='object')


,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab
568892,NaN,1,0,NaN
636060,NaN,1,0,NaN
1398343,NaN,1,0,NaN
398126,NaN,1,0,NaN
808662,NaN,1,0,NaN


In [21]:
# eliminando duplicados
FeatEstab = FeatEstab.drop_duplicates()
print(FeatEstab.shape)
FeatEstab.sort_values(by=['codEstab']).head()

(74339, 4)


,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab
398126,NaN,1,0,NaN
9507,NaN,2,0,NaN
142715,100.0,3,1,166.0
568696,100.0,4,1,401.0
1250161,100.0,5,1,175.0
